<a href="https://colab.research.google.com/github/antonioalbanese/Time-Series-Anomaly-Detection-An-experimental-survey/blob/main/Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install wandb --quiet
# !wandb login

# import wandb


!git clone https://github.com/antonioalbanese/Time-Series-Anomaly-Detection-An-experimental-survey/

%cd Time-Series-Anomaly-Detection-An-experimental-survey

# %mkdir dataset
# !gdown https://drive.google.com/uc?id=1ZCLBU_pKTbsPlcj_LwxZE3IRy6mrlys3 -O="./dataset/MSL.zip"
# !gdown https://drive.google.com/uc?id=18JNYBsaX7tu0Qfgo92nCBCklv471L1xB -O="./dataset/SMD.zip"
# import zipfile
# with zipfile.ZipFile("./dataset/MSL.zip", 'r') as zip_ref:
#     zip_ref.extractall("./dataset/")
# with zipfile.ZipFile("./dataset/SMD.zip", 'r') as zip_ref:
#     zip_ref.extractall("./dataset/")

Cloning into 'Time-Series-Anomaly-Detection-An-experimental-survey'...
remote: Enumerating objects: 819, done.
remote: Counting objects: 100% (302/302), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 819 (delta 207), reused 203 (delta 143), pack-reused 517
Receiving objects: 100% (819/819), 2.75 MiB | 11.09 MiB/s, done.
Resolving deltas: 100% (480/480), done.
/content/Time-Series-Anomaly-Detection-An-experimental-survey


In [2]:
!mkdir data
!mkdir data/SWAT
#normal period
!python USAD/gdrivedl.py https://drive.google.com/open?id=1rVJ5ry5GG-ZZi5yI4x9lICB8VhErXwCw data/SWAT
#anomalies
!python USAD/gdrivedl.py https://drive.google.com/open?id=1iDYc0OEmidN712fquOBRFjln90SbpaE7 data/SWAT

data/SWAT/SWaT_Dataset_Normal_v1.csv
[==================================================] 163.77MB/163.77MB
data/SWAT/SWaT_Dataset_Attack_v0.csv
[==================================================] 127.27MB/127.27MB


In [8]:
### NEW DEEPANT
from AnomalyDetectionMethodClass import ADMethod

configuration = {
    'DATASET': 'SWAT',
    'DATAPATH': None,
    'SEQ_LEN': 100,
    'STEP': 100,
    'LR': 0.0001,
    'EPOCHS': 50,
    'VERBOSE': True
}
method = ADMethod(name = 'DEEPANT', config = configuration)

/content/Time-Series-Anomaly-Detection-An-experimental-survey/datafactory.py:20: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  self.load_data()
/content/Time-Series-Anomaly-Detection-An-experimental-survey/datafactory.py:20: DtypeWarning: Columns (1,9,28,46) have mixed types.Specify dtype option on import or set low_memory=False.
  self.load_data()
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [9]:
method.train()

Epoch 1/50: train_loss:0.5505902618169785
Epoch 2/50: train_loss:0.5411247342824936
Epoch 3/50: train_loss:0.5226950854063034
Epoch 4/50: train_loss:0.4948487788438797
Epoch 5/50: train_loss:0.4588527485728264
Epoch 6/50: train_loss:0.39776845574378966
Epoch 7/50: train_loss:0.3208754763007164
Epoch 8/50: train_loss:0.3099981516599655
Epoch 9/50: train_loss:0.3469611406326294
Epoch 10/50: train_loss:0.2684433996677399
Epoch 11/50: train_loss:0.3942651227116585
Epoch 12/50: train_loss:0.23967766836285592
Epoch 13/50: train_loss:0.21334592029452323
Epoch 14/50: train_loss:0.19394998028874397
Epoch 15/50: train_loss:0.19993529841303825
Epoch 16/50: train_loss:0.18783982917666436
Epoch 17/50: train_loss:0.17277236580848693
Epoch 18/50: train_loss:0.16105403304100036
Epoch 19/50: train_loss:0.20590783432126045
Epoch 20/50: train_loss:0.17760936617851258
Epoch 21/50: train_loss:0.18636239543557168
Epoch 22/50: train_loss:0.19816538766026498
Epoch 23/50: train_loss:0.18937015682458877
Epoch 2

nuovo metodo training sembra funzionare con SWAT, implementare test e poi altri dataset su Deep Ant. Poi riportare nel nuovo metodo tutti gli altri modelli

In [12]:
import torch
t = torch.tensor([
               [
                 [1,11],
                 [2,22],
                 [3,33]
               ],
               [
                 [4,44],
                 [5,55],
                 [6,66]
               ],
               [
                 [7,77],
                 [8,88],
                 [9,99]
                ],
               [
                 [10,1010],
                 [11,1111],
                 [12,1212]
                ],
               [
                 [13,1313],
                 [14,1414],
                 [15,1515]
                ]
            ])

print(t.size())
#256, 100, 51
tp = t.permute(0,-1,1)
print(tp)

tp.size()


torch.Size([5, 3, 2])
tensor([[[   1,    2,    3],
         [  11,   22,   33]],

        [[   4,    5,    6],
         [  44,   55,   66]],

        [[   7,    8,    9],
         [  77,   88,   99]],

        [[  10,   11,   12],
         [1010, 1111, 1212]],

        [[  13,   14,   15],
         [1313, 1414, 1515]]])


torch.Size([5, 2, 3])

In [ ]:
### USAD
from AnomalyDetectionMethodClass import ADMethod
settings={
    'EPOCHS':20,
    'BATCH_SIZE': 7919,
    'SEQ_LEN': 12,
    'HIDDEN_SIZE': 100,
    'ALPHA': 0.5,
    'BETA': 0.5,
    'CONFIDENCE': 0.82,
    'DATASET': 'SWAT'}
data_path = ''
method = ADMethod('USAD',settings,'')

method.prepare_pipeline()
h = method.train()
scores, report = method.test()

In [ ]:
###  TRANSFORMER
from AnomalyDetectionMethodClass import ADMethod
data_path = './dataset/MSL'
settings = {
    'lr':1e-4,
    'num_epochs':3,
    'k':3,
    'win_size':100,
    'input_c':55,
    'output_c':55,
    'batch_size':256,
    'pretrained_model':None,
    'dataset':'MSL',
    'mode':'train', # choices=['train', 'test']
    'data_path':'./dataset/MSL',
    'model_save_path':'checkpoints',
    'anormly_ratio':1
}
method = ADMethod('transformer',settings, data_path)

method.prepare_pipeline()
method.train()
scores, report = method.test()

In [ ]:
### DEEPANT
from AnomalyDetectionMethodClass import ADMethod
# data_path = './NAB/realKnownCause/cpu_utilization_asg_misconfiguration.csv'
data_path = './dataset/MSL'


LR_LIST = [0.001, 0.0001, 0.00001, 0.000001]
SEQ_LIST = [100, 30, 100, 300]
for lr in [0.0001]:#LR_LIST:
  for seq in [100]: #SEQ_LIST:
    # wandb.init(project="experimental-survey-AD", entity="michiamoantonio", group='DeepAnt')
    # wandb.log({'lr': lr})
    # wandb.log({'seq_len': seq})
    settings = {
      'SEQ_LEN': seq,
      'out_dim': 1,
      'EPOCHS': 1,
      'LR': lr,
      'CONFIDENCE': 0.50,
      'TH_LIST': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
      'TH_SEARCH': True,
      'VERBOSE': True,
      'DATASET': 'SWAT'
    }
    # wandb.config = settings
    method = ADMethod('DeepAnt',settings, data_path)

    method.prepare_pipeline()
    l = method.train()
    scores, report = method.test()

    print(f"lr: {lr}, seq_len: {seq}")
    for i,el in enumerate(report): 
      print(f"#### th: {settings['TH_LIST'][i]}, f1-score: {el['macro avg']['f1-score']}")
      # wandb.log({'th': settings['TH_LIST'][i], 'f1-score': el['macro avg']['f1-score']})
    # wandb.finish()



LOAD TRAIN DATA


/content/Time-Series-Anomaly-Detection-An-experimental-survey/DeepAnt/DeepAntSolver.py:22: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  self.prepare_data(data_path)


SCALER
SCALED
window
WINDOWED


In [ ]:
!pip install torchinfo
from torchinfo import summary
summary(method.solver.model)